# Alzheimer textual explanation, visual explanation and classification
In this notebook there's all the procedure we do for the classification and for the explanation.

For the realization of this project i start from the code of my colleague.

In this notebook we suppose that you have already the dataset and the explanation, if else, 
you will run "Creation of the dataset" before this notebook.

In [10]:
import os, random, glob
import pickle

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sys import platform
import re
import html
import string
import unicodedata
from nltk.tokenize import word_tokenize
import os
from tqdm.notebook import tqdm

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences, to_categorical, plot_model 
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Input, Dense,
    LSTM, Embedding,
    Dropout, add,
    MaxPool3D, Conv3D,
    GlobalAveragePooling3D, BatchNormalization
)

import torch
from torch.utils.data import Dataset, DataLoader

from PIL import Image

from alzheimer_disease.src.modules.postprocessing import get_gradcam, get_affected_areas
from alzheimer_disease.src.helpers.utils import get_device
from alzheimer_disease.src.modules.training import training_model
from alzheimer_disease.src.helpers.config import get_config
from alzheimer_disease.src.modules.preprocessing import get_transformations
from alzheimer_disease.src.models.densenetmm import DenseNetMM

In [11]:
# Definition of all paths
dataset = 'oasis_aug'

_base_path = '/Volumes/Seagate Bas/Vito/CV'
_config = get_config()
saved_path = os.path.join(_base_path, _config.get('SAVED_FOLDER'))
reports_path = os.path.join(_base_path, _config.get('REPORT_FOLDER'))
logs_path = os.path.join(_base_path, _config.get('LOG_FOLDER'))
_data_path = os.path.join(_base_path, _config.get('LOCAL_DATA'))
data_path, meta_path, explanation_path = [
    os.path.join(_data_path, dataset, 'data/'),
    os.path.join(_data_path, dataset, 'meta/'),
    os.path.join(_data_path, dataset, 'explainability/')
]

device = get_device()

if platform == 'win32':
    saved_path = saved_path.replace('/', '\\')
    reports_path = reports_path.replace('/', '\\')
    logs_path = logs_path.replace('/', '\\')
    data_path = data_path.replace('/', '\\')
    meta_path = meta_path.replace('/', '\\')
    explanation_path = explanation_path.replace('/', '\\')

saved_path, reports_path, logs_path, data_path, meta_path, explanation_path, device

('/Volumes/Seagate Bas/Vito/CV/saved/',
 '/Volumes/Seagate Bas/Vito/CV/reports/',
 '/Volumes/Seagate Bas/Vito/CV/logs/',
 '/Volumes/Seagate Bas/Vito/CV/data/oasis_aug/data/',
 '/Volumes/Seagate Bas/Vito/CV/data/oasis_aug/meta/',
 '/Volumes/Seagate Bas/Vito/CV/data/oasis_aug/explainability/',
 'cpu')

In [12]:
SIZE = 128
output_length = 1024
epochs = 30

CHANNELS = ['T2w']

FEATURES = ['sex', 'age', 'bmi', 'education', 'cdr_memory', 'cdr_orientation', 'cdr_judgment', 'cdr_community', 'cdr_hobbies', 'cdr_personalcare', 'boston_naming_test', 'depression', 'sleeping_disorder', 'motor_disturbance']
MULTICLASS = True

In [13]:
# I started with the train test split of colleague and adapt to my task
def train_test_splitting(
        data_folder,
        meta_folder,
        explanation_folder,
        channels,
        features,
        train_ratio=.8,
        multiclass=False,
        verbose=True
):
    """
    Splitting train/eval/test.
    Args:
        data_folder (str): path of the folder containing images.
        meta_folder (str): path of the folder containing csv files.
        explanation_folder (str): path of the folder containing csv files of the explanation.
        channels (list): image channels to select (values `T1w`, `T2w` or both).
        features (list): features set to select.
        train_ratio (float): ratio of the training set, value between 0 and 1.
        multiclass (bool): `False` for binary classification, `True` for ternary classification.
        verbose (bool): whether or not print information.
    Returns:
        train_data (list): the training data ready to feed monai.data.Dataset
        eval_data (list): the evaluation data ready to feed monai.data.Dataset
        test_data (list): the testing data ready to feed monai.data.Dataset.
        (see https://docs.monai.io/en/latest/data.html#monai.data.Dataset).
    """
    scaler = MinMaxScaler()
    df = pd.read_csv(os.path.join(meta_folder, 'data_num.csv'))
    df1 = df[(df['weight'] != .0) & (df['height'] != .0)]
    df['bmi'] = round(df1['weight'] / (df1['height'] * df1['height']), 0)
    df['bmi'] = df['bmi'].fillna(.0)
    sessions = [s.split('_')[0] for s in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, s))]
    subjects = list(set(sessions))

    # uploading of the dataset
    explanation = pd.read_csv(explanation_folder + 'explaination.csv', sep=';')



    # applying splitting on subjects to prevent data leakage
    random.shuffle(subjects)
    split_train = int(len(subjects) * train_ratio)
    train_subjects, test_subjects = subjects[:split_train], subjects[split_train:]
    split_eval = int(len(train_subjects) * .8)
    eval_subjects = train_subjects[split_eval:]
    train_subjects = train_subjects[:split_eval]

    # applying multiclass label correction and splitting
    if multiclass:
        train_subjects, eval_subjects, test_subjects = [], [], []
        df.loc[df['cdr'] == .0, 'final_dx'] = .0
        df.loc[df['cdr'] == .5, 'final_dx'] = 1.
        df.loc[(df['cdr'] != .0) & (df['cdr'] != .5), 'final_dx'] = 2.
        m = np.min(np.unique(df['final_dx'].to_numpy(), return_counts=True)[1])
        df = pd.concat([
            df[df['final_dx'] == .0].sample(m),
            df[df['final_dx'] == 1.].sample(m),
            df[df['final_dx'] == 2.].sample(m)
        ], ignore_index=True)
        n_test = m - int(m * train_ratio)
        n_eval = m - n_test - int(m * train_ratio * train_ratio)
        for i in range(3):
            sub = list(set(df[df['final_dx'] == float(i)]['subject_id'].to_numpy()))
            random.shuffle(sub)
            counter = 0
            for j in range(len(sub)):
                counter += len(df[df['subject_id'] == sub[j]])
                if counter <= n_test:
                    test_subjects.append(sub[j])
                elif counter > n_test and counter <= (n_test + n_eval):
                    eval_subjects.append(sub[j])
                else:
                    train_subjects.append(sub[j])

    # loading sessions paths
    X_train = df[df['subject_id'].isin(train_subjects)]
    X_eval = df[df['subject_id'].isin(eval_subjects)]
    X_test = df[df['subject_id'].isin(test_subjects)]
    train_sessions = [os.path.join(data_folder, s) for s in X_train['session_id'].values]
    eval_sessions = [os.path.join(data_folder, s) for s in X_eval['session_id'].values]
    test_sessions = [os.path.join(data_folder, s) for s in X_test['session_id'].values]

    # loading explanation of subjects
    explanation_train = explanation[explanation['subject_id'].isin(X_train['subject_id'].values)]
    explanation_eval = explanation[explanation['subject_id'].isin(X_eval['subject_id'].values)]
    explanation_test = explanation[explanation['subject_id'].isin(X_test['subject_id'].values)]

    # scaling numerical data in range [0,1]
    X_train.loc[:, features] = scaler.fit_transform(X_train[features])
    X_eval.loc[:, features] = scaler.fit_transform(X_eval[features])
    X_test.loc[:, features] = scaler.fit_transform(X_test[features])

    # arranging data in dictionaries
    # I will also take the reference session of the explanation and the image
    train_data = [dict({
        'image': sorted([os.path.join(s, i) for i in os.listdir(s) if any(c in i for c in channels)]),
        'data': X_train[X_train['session_id'] == s.split('/')[-1]][features].values[0],
        'label': df[df['session_id'] == s.split('/')[-1]]['final_dx'].values[0],
        'explanation': explanation_train[explanation_train['session_id'] == s.split('/')[-1]]['explaination'].values[0],
        'session_id': s.split('/')[-1]
    }) for s in train_sessions]
    eval_data = [dict({
        'image': sorted([os.path.join(s, i) for i in os.listdir(s) if any(c in i for c in channels)]),
        'data': X_eval[X_eval['session_id'] == s.split('/')[-1]][features].values[0],
        'label': df[df['session_id'] == s.split('/')[-1]]['final_dx'].values[0],
        'explanation': explanation_eval[explanation_eval['session_id']==s.split('/')[-1]]['explaination'].values[0],
        'session_id': s.split('/')[-1]
    }) for s in eval_sessions]
    test_data = [dict({
        'image': sorted([os.path.join(s, i) for i in os.listdir(s) if any(c in i for c in channels)]),
        'data': X_test[X_test['session_id'] == s.split('/')[-1]][features].values[0],
        'label': df[df['session_id'] == s.split('/')[-1]]['final_dx'].values[0],
        'explanation': explanation_test[explanation_test['session_id'] == s.split('/')[-1]]['explaination'].values[0],
        'session_id': s.split('/')[-1]
    }) for s in test_sessions]

    # print data splitting information
    if verbose:
        print(''.join(['> ' for _ in range(40)]))
        print(f'\n{"":<20}{"TRAINING":<20}{"EVALUATION":<20}{"TESTING":<20}\n')
        print(''.join(['> ' for _ in range(40)]))
        tsb1 = str(len(train_subjects)) + ' (' + str(round((len(train_subjects) * 100 / len(df['subject_id'].unique())), 0)) + ' %)'
        tsb2 = str(len(eval_subjects)) + ' (' + str(round((len(eval_subjects) * 100 / len(df['subject_id'].unique())), 0)) + ' %)'
        tsb3 = str(len(test_subjects)) + ' (' + str(round((len(test_subjects) * 100 / len(df['subject_id'].unique())), 0)) + ' %)'
        tss1 = str(len(train_sessions)) + ' (' + str(round((len(train_sessions) * 100 / len(df)), 2)) + ' %)'
        tss2 = str(len(eval_sessions)) + ' (' + str(round((len(eval_sessions) * 100 / len(df)), 2)) + ' %)'
        tss3 = str(len(test_sessions)) + ' (' + str(round((len(test_sessions) * 100 / len(df)), 2)) + ' %)'
        print(f'\n{"subjects":<20}{tsb1:<20}{tsb2:<20}{tsb3:<20}\n')
        print(f'{"sessions":<20}{tss1:<20}{tss2:<20}{tss3:<20}\n')

    return train_data, eval_data, test_data

In [14]:
densenet = DenseNetMM(
    in_channels = len(CHANNELS),
    in_size = SIZE,
    in_features_size= len(FEATURES),
    out_channels = 3 if MULTICLASS else 2,
    append_features = True
)

In [15]:
train_transform, eval_transform = get_transformations(size=SIZE)

train, val, test = train_test_splitting(
    data_folder=data_path,
    meta_folder=meta_path,
    explanation_folder=explanation_path,
    channels=CHANNELS,
    features=FEATURES,
    multiclass=MULTICLASS,
    verbose=True
)

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 

                    TRAINING            EVALUATION          TESTING             

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 

subjects            381 (64.0 %)        94 (16.0 %)         118 (20.0 %)        

sessions            440 (64.33 %)       107 (15.64 %)       137 (20.03 %)       



In [16]:
train[0]

{'image': ['/Volumes/Seagate Bas/Vito/CV/data/oasis_aug/data/OAS31302_MR_d0056/sub-OAS31302_sess-d0056_T2w.nii.gz'],
 'data': array([1.        , 0.47916667, 0.63043478, 0.66666667, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 1.        , 1.        , 0.5       ]),
 'label': 0.0,
 'explanation': "**Summary of Heatmap Analysis**\n\nThe heatmap analysis reveals that the classification model focused on specific regions in the brain, which are not areas typically affected by Alzheimer's Disease. The following regions were highlighted in the heatmap:\n\n1. **Frontal-to-Occipital (GapMap) left**: This region accounted for 19.9% of the heatmap and 17.81% of the affected region. The GapMap region is responsible for processing visual information and integrating it with other sensory inputs. The model's focus on this region may indicate that the patient's visual processing abilities were affected, potentially contributing to the diagnosis of A

In [17]:
if glob.glob(saved_path+ 'DenseNetMM_best.pth'):
	print('Loading DenseNetMM_best.pth')
	densenet.load_state_dict(torch.load(saved_path + 'DenseNetMM_best.pth'))
else:
	print('Train of the model')
	train_metrics = training_model(
		model = densenet,
		data = [train, val],
		transforms = [train_transform, eval_transform],
		epochs = epochs,
		device = get_device(),
		paths = [saved_path, reports_path, logs_path],
		num_workers=0,
		verbose=True
	)

Loading DenseNetMM_best.pth


## Image Captioning

In [34]:
name_fextractor = 'DenseNetMMFeatureExtractor'

In [35]:
from monai.data import CacheDataset


def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\n', "\n").replace('quot;', "'").replace(
        '', "\n").replace('\"', '"').replace('', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\'', ' \ ')
    return re1.sub(' ', html.unescape(x1))


def remove_non_ascii(text):
    """Remove non-ASCII characters from list of tokenized words"""
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')


def to_lowercase(text):
    return text.lower()

def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):
    """
    :param words:
    :type words:
    :param stop_words: from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
    or
    from spacy.lang.en.stop_words import STOP_WORDS
    :type stop_words:
    :return:
    :rtype:
    """
    return [word for word in words if word not in stop_words]

def addsequences(text):
  return '' .join('startseq ' + " ".join([word for word in text.split() if len(word)>1]) + ' endseq')

def text2words(text):
    return word_tokenize(text)

def normalize_text( text):
    text = remove_special_chars(text)
    text = remove_non_ascii(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    text = addsequences(text)
    return text
  
def normalize_corpus(corpus):
    return [normalize_text(t) for t in corpus]

def idx_to_word(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

# concat for get all df
entire_df = train + val + test

all_text = [explaination['explanation'] for explaination in entire_df]

# tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(all_text))
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: {}'.format(vocab_size))

Vocabulary Size: 1154


In [36]:
feature_extractor = DenseNetMM(
    in_channels = len(CHANNELS),
    in_size = SIZE,
    in_features_size= len(FEATURES),
    out_channels = 3 if MULTICLASS else 2,
    append_features = False,
    name=name_fextractor
)

if glob.glob(saved_path+ f'{name_fextractor}.pth'):
	print('Loading DenseNetMM_best.pth for feature extraction')
	feature_extractor.load_state_dict(torch.load(saved_path + 'DenseNetMM_best.pth'))
else:
	print('Train of the model for feature extraction')
	train_metrics = training_model(
		model = feature_extractor,
		data = [train, val],
		transforms = [train_transform, eval_transform],
		epochs = epochs,
		device = get_device(),
		paths = [saved_path, reports_path, logs_path],
		num_workers=0,
		verbose=True
	)


# restructure the model    
feature_extractor = Model(inputs=feature_extractor.inputs, outputs=feature_extractor.layers[-2].output)
# summarize
print(feature_extractor.summary())

Train of the model for feature extraction
> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 
epoch 1/30


KeyboardInterrupt: 

In [32]:
for i in feature_extractor.named_children():
    print(i)

('features_img', Sequential(
  (conv0): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
  (norm0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU(inplace=True)
  (pool0): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (denseblock1): _DenseBlock(
    (denselayer1): _DenseLayer(
      (layers): Sequential(
        (norm1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (norm2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv3d(128, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      )
    )
    (denselayer2): _DenseLayer(
      (layers): Sequential(
        (norm1): BatchNorm3d(96, eps=1e-05, m

In [ ]:
# extract features from image
features = {}
entire_df = CacheDataset(entire_df, transform=eval_transform, cache_rate=1.0, num_workers=None, progress=False)
entire_df = DataLoader(entire_df, batch_size=1, shuffle=False, num_workers=0)

for img_name in tqdm(entire_df):
    val_inputs_img, val_inputs_data, session_id  = (
        img_name['image'].to(device),
        img_name['data'].to(device),
        img_name['session_id'].to(device)
    )
    
    # extract features
    feature = feature_extractor.predict([val_inputs_img, val_inputs_data])
    # store feature
    features[session_id] = feature
features

In [ ]:
pickle.dump(features, open(os.path.join(saved_path, 'clef_features_vgg.pkl'), 'wb'))

In [ ]:
if glob.glob(saved_path + 'clef_features_vgg.pkl'):
    with open(os.path.join(saved_path, 'clef_features_vgg.pkl'), 'rb') as f:
        features = pickle.load(f)

In [ ]:
# encoder model
# image feature layers
inputs1 = Input((128, 128, 50, 1))

encoding = Conv3D(filters=64, kernel_size=3, activation="relu")(inputs1)
encoding = MaxPool3D(pool_size=2)(encoding)
encoding = BatchNormalization()(encoding)

encoding = Conv3D(filters=64, kernel_size=3, activation="relu")(encoding)
encoding = MaxPool3D(pool_size=2)(encoding)
encoding = BatchNormalization()(encoding)

encoding = Conv3D(filters=128, kernel_size=3, activation="relu")(encoding)
encoding = MaxPool3D(pool_size=2)(encoding)
encoding = BatchNormalization()(encoding)

encoding = Conv3D(filters=256, kernel_size=3, activation="relu")(encoding)
encoding = MaxPool3D(pool_size=2)(encoding)
encoding = BatchNormalization()(encoding)

encoding = GlobalAveragePooling3D()(encoding)
encoding = Dense(units=512, activation="relu")(encoding)
encoding = Dropout(0.3)(encoding)

# sequence feature layers
inputs2 = Input(shape=(output_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.4)(se1)
se3 = LSTM(256)(se2)

# decoder model
decoder = add([encoding, se3])
decoder = Dense(256, activation='relu')(decoder)
outputs = Dense(vocab_size, activation='softmax')(decoder)

generation_model = Model(inputs=[inputs1, inputs2], outputs=outputs)
generation_model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["accuracy"])

# plot the model
plot_model(generation_model, show_shapes=True)

In [ ]:
# create data generator to get data in batch (avoids session crash)
def data_generator(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
    # loop over images
    X1, X2, y = list(), list(), list()
    n = 0
    while 1:
        for key in data_keys:
            n += 1
            feature_key = key.split('.')[0]
            captions = mapping[key]
            # process each caption
            for caption in captions:
                # encode the sequence
                seq = tokenizer.texts_to_sequences([caption])[0]
                # split the sequence into X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pairs
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    
                    # store the sequences
                    #print([key])
                    X1.append(features[feature_key][0])
                    X2.append(in_seq)
                    y.append(out_seq)
            if n == batch_size:
                X1, X2, y = np.array(X1), np.array(X2), np.array(y)
                yield [X1, X2], y
                X1, X2, y = list(), list(), list()
                n = 0

In [ ]:
# train the model
batch_size = 32
steps = len(train) // batch_size

train_generator = data_generator(train, mapping, features, tokenizer, output_length, vocab_size, batch_size)
val_generator = data_generator(val, mapping, features, tokenizer, output_length, vocab_size, batch_size)
# fit for 10 epoch
generation_model.fit(train_generator, epochs=epochs,validation_data=val_generator, steps_per_epoch=steps, verbose=1)
generation_model.save(saved_path+'/model_generation.h5')

In [ ]:
def predict_caption(model, image, tokenizer, max_length):
    # add start tag for generation process
    in_text = 'startseq'
    # iterate over the max length of sequence
    for i in range(max_length):
        # encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad the sequence
        sequence = pad_sequences([sequence], max_length)
        # predict next word
        yhat = model.predict([image, sequence], verbose=0)
        # get index with high probability
        yhat = np.argmax(yhat)
        # convert index to word
        word = idx_to_word(yhat, tokenizer)
        # stop if word not found
        if word is None:
            break
        # append word as input for generating next word
        in_text += " " + word
        # stop if we reach end tag
        if word == 'endseq':
            break
      
    return in_text

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
# validate with test data
actual, predicted = list(), list()
all_captions = []

for key in tqdm(test):
    # get actual caption
    feature_key = key.split('.')[0]
    captions = mapping[key]
    # predict the caption for image
    y_pred = predict_caption(generation_model, features[feature_key], tokenizer, output_length)
    all_captions.append(y_pred)
    # split into words
    actual_captions = [caption.split() for caption in captions]
    y_pred = y_pred.split()
    # append to the list
    actual.append(actual_captions)
    predicted.append(y_pred)
    
# calcuate BLEU score
print("BLEU-1: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
print("BLEU-2: %f" % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
print(all_captions[0]) 